---
# Chapter 8
## Object References, Mutability, and Recycling

---

## Variables are not boxes

---
### Example 8-1: Variables *a* and *b* hold references to the same list, not copies of the list

In [3]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

---
### Example 8-2: Variables are asigned to objects only after the objects are created

In [7]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))


x = Gizmo()


Gizmo id: 4586724416


In [9]:
y = Gizmo() * 10

Gizmo id: 4586725280


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [11]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_10',
 '_2',
 '_3',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'os',
 'quit',
 'sys',
 'x']

## Identity, Equality, and Aliases

---
### Example 8-3: charles and lewis refer to the same object

In [12]:
charles = {'name': 'Charles L. Dogson', 'born': 1832}
lewis = charles

In [13]:
lewis is charles

True

In [14]:
id(charles), id(lewis)

(4598784832, 4598784832)

In [15]:
lewis['balance'] = 950

In [16]:
charles

{'name': 'Charles L. Dogson', 'born': 1832, 'balance': 950}

---
### Example 8-4: alex and charles compare equal, but alex is not charles

In [18]:
alex = {'name': 'Charles L. Dogson', 'born': 1832, 'balance': 950}
alex == charles

True

In [19]:
alex is charles

False

## The relative immutability of Tuples

---
### Example 8-5: t1 and t2 initially compare equal, but changing a mutable item inside tuple t1 makes it different

In [20]:
t1 = (1, 2, [20, 40])
t2 = (1, 2, [20, 40])

In [21]:
t1 == t2

True

In [22]:
id(t1[-1])

4598891840

In [23]:
t1[-1].append(99)
t1

(1, 2, [20, 40, 99])

In [24]:
id(t1[-1])

4598891840

In [25]:
t1 == t2

False

## Copies Are Shallow by Default

---
### Example 8-6: Making a shallow copy of a list containng another list; copy and paste this code to see it animated at the Online Python Tutor ([https://pythontutor.com](https://pythontutor.com))

In [26]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)      # 
l1.append(100)     # 
l1[1].remove(55)   # 
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]  # 
l2[2] += (10, 11)  # 
print('l1:', l1) 

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]


## Deep and Shallow Copies of Arbitrary Objects

---
### Example 8-8: Bus pickl up and drops off passengers

In [28]:
class Bus:

    def __init__(self, passangers=None):
        if passangers is None:
            self.passangers = []
        else:
            self.passangers = list(passangers)

    def pick(self, name):
        self.passangers.append(name)

    def drop(self, name):
        self.passangers.remove(name)


---
### Example 8-9: Effects of using copy versus deepcopy

In [32]:
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print('id bus 1: {0}, id bus 2: {1}, id bus3: {2}'.format(id(bus1), id(bus2), id(bus3)))
bus1.drop('Bill')
print(bus2.passangers)
print('id passanger bus 1: {0}, id passanger bus 2: {1}, id passanger bus3: {2}'.format(id(bus1.passangers), id(bus2.passangers), id(bus3.passangers)))
print(bus3.passangers)

id bus 1: 4598223152, id bus 2: 4598225024, id bus3: 4598223008
['Alice', 'Claire', 'David']
id passanger bus 1: 4598352064, id passanger bus 2: 4598352064, id passanger bus3: 4598800128
['Alice', 'Bill', 'Claire', 'David']


---
### Example 8-10: Cyclic referencesL b refers to a, and then is appended to a: deecopy still manages to copy a

In [33]:
a = [10, 20]
b = [a, 30]
a.append(b)
print(a)

from copy import deepcopy
c = deepcopy(a)
print(c)

[10, 20, [[...], 30]]
[10, 20, [[...], 30]]


## Function Parameters as References

---
### Example 8-11: A function may change any mutable object it receives

In [40]:
def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x, y)

3

In [41]:
x, y

(1, 2)

In [42]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [43]:
a, b

([1, 2, 3, 4], [3, 4])

In [44]:
t = (10, 20,)
u = (30, 40,)
f(t, u)

(10, 20, 30, 40)

In [45]:
t, u

((10, 20), (30, 40))

## Mutable Types as Parameter Defaults: Bad Idea

---
### Example 8-12: A simple class to illustrate the danger of a mutable  default

In [77]:
class HauntedBus:
    """A bus model haunted by ghot passengers"""
    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name: str) -> None:
        self.passengers.append(name)

    def drop(self, name: str) -> None:
        self.passengers.remove(name)

---
### Example 8-13: Buses haunted by ghost passengers

In [78]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [79]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [80]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [81]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [82]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [83]:
bus2.passengers is bus3.passengers

True

In [84]:
bus1.passengers

['Bill', 'Charlie']

In [85]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [87]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [88]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

## Defensive Programming with Mutable Parameters

---
### Example 8-14: Passengers disappear when dropped by a TwilightBus

In [94]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

---
### Example  8-15: A simple class to show the perils of mutating received arguments

In [97]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

## del and Garbage Collection

---
### Example 8-16: Watching the end of an object when no more references point to it

In [99]:
import  weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)
s2 = "spam"
print(ender.alive)

True
True
Gone with the wind...
False


## Weak References

---
### Example 8-17: A weak reference is a callable that returns the references object or None if the referencet is not more

In [9]:
import weakref

a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x1086bc860; to 'set' at 0x1086d9040>

In [10]:
wref()

{0, 1}

In [11]:
a_set = {2, 3, 4}
wref()

{0, 1}

In [12]:
wref() is None

False

In [13]:
wref() is None

False

## The WeakValueDictionary Skit

---
### Example 8-18:  Cheese has a kind attribute and standard representation

In [104]:
class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

---
### Example 8-19: Customer: Have you in fact got any cheese here at all?"

In [107]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese

print(sorted(stock.keys()))

del catalog
del cheese
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
[]


## Limitations of Weak References

In [14]:
class MyList(list):
    """asdasd"""
    pass

a_list  = MyList(range(10))

wref_to_a_list = weakref.ref(a_list)

## Trics Python Plays with Immutables

### Example 8-20: A tuple built from another is actually the same exact tuple

In [15]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t2

True

In [16]:
t3 = t1[:]
t3 is t1

True

### Example 8-21: String literals may create shared objects

In [17]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [18]:
s1 = "ABC"
s2 = "ABC"
s2 is s1

True